## Import Packages

In [1]:
!pip install gurobipy>=9.5.1

zsh:1: 9.5.1 not found


In [2]:
import gurobipy as gp
from gurobipy import GRB as GRB
import sys
sys.path.insert(0, "../utils")

import column_generation
from unavailability import optimal_group1_unavailability

## Column Generation

In [3]:
generic_columns = column_generation.generic_column_generation(5)
group1_feasible_combinations = column_generation.group_column_generation(1, optimal_group1_unavailability, generic_columns)

Total possible combinations: 1048576
Total feasible combinations: 45046
Group 1 Unavailability Mapping: {1: [10], 2: [6, 10], 3: [10], 4: [10], 5: [2, 3, 4, 8, 9, 10, 14, 16, 19], 6: [10], 7: [], 8: [7, 18], 9: [], 10: [4, 7, 16], 11: [0, 4, 5, 6, 7, 12, 13, 15, 16], 12: [4, 6, 7, 16], 13: [4, 6, 7, 12, 15, 16], 14: [1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 14, 15, 16, 19], 15: [2, 3, 4, 6, 7, 8, 9, 12, 14, 15, 16, 19], 16: [4, 6, 12, 15, 16], 17: [4, 6, 7, 12, 15, 16, 18], 18: [4, 6, 7, 12, 15, 16, 18], 19: [], 20: [0, 2, 3, 4, 5, 8, 9, 12, 13, 14, 15, 16, 19], 21: [0, 5, 12, 13, 15], 22: [], 23: [1, 7, 11, 18], 24: [1, 11], 25: [], 26: [0, 5, 12, 13, 15], 27: [], 28: [5], 29: [1, 5, 11], 30: [5], 31: [5], 32: [5], 33: [5], 34: [], 35: [10, 19], 36: [6, 10, 19], 37: [10, 19], 38: [10, 19], 39: [2, 3, 4, 8, 9, 10, 14, 16, 19], 40: [10, 19], 41: [3], 42: [3, 7, 18], 43: [3], 44: [3], 45: [0, 3, 5, 6, 12, 13, 15], 46: [3, 6], 47: [12], 48: [1, 2, 3, 4, 8, 9, 11, 12, 14, 16, 19], 49: [2, 3, 4, 8, 

## Optimization Model

In [4]:
# Setup Gurobi key
# Create environment with WLS license
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', '453f2da9-b8f2-448f-9f06-48081a9c3dc9')
e.setParam('WLSSECRET', 'd5675c81-7058-45f9-b6f2-30ca3855ed22')
e.setParam('LICENSEID', 868440)
e.start()

# Create the model within the Gurobi environment. This environment will be passed in as a parameter into all subsequent models
model = gp.Model(env=e)
model.setParam('Seed', 435)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 868440
Academic license 868440 - for non-commercial use only - registered to es___@uwaterloo.ca
Set parameter Seed to value 435


In [5]:
# Sets
D = range(1,6)
T = range(1,171)
R = range(1,170)
T_d = {d : (T[(d-1)*len(T)//len(D):(d)*len(T)//len(D)])for d in D}
C = range(1,6)
M = range(6,21)
I = (*C,*M)
H_t = {t : range(1,len(group1_feasible_combinations[t-1])) for t in T}

In [6]:
# penalties
# separated to make them easier to read
# taken from this graph: https://www.desmos.com/calculator/g5rhs6n2ke
penalties = [(0,0), (4,4), (8, 11.31), (12, 20.78), (16, 32), (34, 99.13)]
Q = range(1,len(penalties)+1)

In [7]:
# Parameters

# need to sort this one out with column generation

a = group1_feasible_combinations
f = [[sum(a[t-1][h-1]) // 6 for h in H_t[t]] for t in T]
n = 8 # Need to inpute
u = {q : penalties[q-1][1] for q in Q}
v = {q : penalties[q-1][0] for q in Q}
w = .001 # Need to inpute
g = .01

In [8]:
indices = [(t, h) for t in T for h in H_t[t]]

In [9]:
# Decision Varbiables
alpha = model.addVars(indices, vtype=GRB.BINARY, name="alpha")
y = model.addVars(I, D, Q, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name = "y")
z = model.addVars(I, R, vtype=GRB.BINARY, name = "z")

In [10]:
for i in I:
    for d in D:
        model.addSOS(GRB.SOS_TYPE2, [y[i,d,q] for q in Q])

In [11]:
# Objective Value
model.setObjective(sum(f[t-1][h-1]*alpha[t,h] for t in T for h in H_t[t]) - w * sum(u[q] * y[i,d,q] for i in I for d in D for q in Q) - g * sum(z[i, r] for i in I for r in R),sense=GRB.MAXIMIZE)

In [12]:
model.addConstrs(sum(a[r-1][h-1][i-1]*alpha[r,h] for h in H_t[r]) + sum(a[r][h-1][i-1]*alpha[r+1,h] for h in H_t[r+1]) - 1 <= z[i,r] for i in I for r in R)

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model

In [13]:
# Constraints
model.addConstrs(sum(alpha[t,h] for h in H_t[t]) <= 1 for t in T)
model.addConstrs(sum(a[t-1][h-1][i-1]*alpha[t,h] for t in T_d[d] for h in H_t[t]) <= n for i in I for d in D)
model.addConstrs(sum(a[t-1][h-1][i-1]*alpha[t,h] for t in T_d[d] for h in H_t[t]) == sum(v[q]*y[i,d,q] for q in Q) for i in I for d in D)
model.addConstrs(sum(y[i,d,q] for q in Q) == 1 for i in I for d in D)

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 4): <gurobi.Constr *Awaiting Model Update*>,
 (4, 5): <gurobi.Constr *Awaiting Model Update*>,


In [14]:
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 868440 - for non-commercial use only - registered to es___@uwaterloo.ca
Optimize a model with 3850 rows, 2626077 columns and 99339947 nonzeros
Model fingerprint: 0x91567008
Model has 100 SOS constraints
Variable types: 600 continuous, 2625477 integer (2625477 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [4e-03, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 286 rows and 170 columns (presolve time = 5s) ...
Presolve removed 286 rows and 453 columns (presolve time = 10s) ...
Presolve removed 318 rows and 485 columns (presolve time = 16s) ...
Presolve removed 318 rows and 485 columns (presolve time = 21s) ...
Presolve removed 891 rows and 485 columns (presolve time = 27s) ...
Presolve removed 891 rows and 485

In [15]:
filename = "../data/optimal_groups/group1_results.txt"
with open(filename, "w") as file:
  for t in T:
    for h in H_t[t]:
      if alpha[t,h].X == 1:
        file.write(f"{t} {a[t-1][h-1]}\n")
        print(t, a[t-1][h-1])


1 (0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1)
4 (0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0)
6 (0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0)
7 (1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1)
9 (0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0)
10 (0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0)
12 (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1)
13 (0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0)
16 (0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0)
19 (0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1)
21 (0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1)
22 (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0)
24 (0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1)
25 (0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0)
27 (0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0)
30 (0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,

In [16]:
print(sum(alpha[t,h].X * sum(a[t-1][h-1][i-1] for i in I) for t in T for h in H_t[t]) // 6)

120.0
